# The Markov Chain Model

The Markov chain (MC) model is the second model used in our thesis. It is significantly more complex than the simple probabilistic model, however, it is still quite rudimentary in comparison with real-life markets. The model was developed by Hult and Kiessling in their paper *[Algorithmic trading with Markov Chains](https://www.researchgate.net/publication/268032734_ALGORITHMIC_TRADING_WITH_MARKOV_CHAINS)*.

In this model, the limit order book (LOB) is modelled explicitly. There are six event types:

> 1. Buy limit orders 
> 2. Sell limit orders
> 3. Cancel buy orders
> 4. Cancel sell orders
> 5. Buy market orders
> 6. Sell market orders

The arrival of an order results in a state transition in the Markov chain. The transition rates are described in our **[report](https://github.com/KodAgge/Reinforcement-Learning-for-Market-Making/blob/main/Reinforcement%20Learning%20for%20Market%20Making.pdf)**. An example of how the arrival of different orders affect the LOB is shown in the image below.

<div>
    <img src="images/LOBDynamics.png" width=800/>
</div>


Like in the simple probabilistic model we also have:

> * The time _t_ can take integer values between _0_ and _T_.
>
> * The market maker has to quote bid and ask prices every second.
>
> * The market maker can put the bid and ask depths at *max\_quote\_depth* different levels, from _1_ to *max\_quote\_depth* ticks away from the best ask and best bid price respectively.
>
> * The cash process _X<sub>t</sub>_ denotes the market makers cash at time _t_.
>
> * The inventory process _Q<sub>t</sub>_ denotes the market makers inventory at time _t_.
>
> * The value process _V<sub>t</sub>_ denotes the value of the market maker's position at time _t_, that is its cash plus the value of its current inventory.
>
> * The market maker can see the current time _t_ and its inventory before taking an action. However, the inventory (_Q<sub>t</sub>_) is grouped into bins, whose size is decided by the parameter $\kappa$.
>
> * At time _t = T_ the market maker is forced to liquidate its position.

The _tick_ is the smallest tradeable unit of the underlying, for instance $0.01 of AAPL.

Contrary to the simple probabilistic model, it is not possible to derive an analytically optimal strategy in the Markov chain model.

## The Q-learning

After that short introduction, it's time for some reinforcement learning in the form of Q-learning.

We start by importing the needed file.

In [1]:
# import the Q-learning file for the markov chain model
from mc_model_evaluation import *

Now we have to decide on the parameters we want to use for the environment and the hyperparameters we want to use for the Q-learning.

There are some additional parameters in the Markov chain model, see the code snippet below for an explanation of them. However, importantly, we choose a longer episode (trading window) of *T = 100*.

In [2]:
model_params = {
                "dt": 1,                    # the length of the time steps
                "T": 100,                   # the length of the episode
                "num_time_buckets": 100,    # how many bins that should be used for the time
                "kappa": 3,                 # the size of the inventory bins
                "num_levels": 10,           # how many depth levels that should be included in the LOB
                "default_order_size": 5,    # the size of the orders the MM places
                "max_quote_depth": 5,       # how deep the MM can put its quotes
}

We now have to decide which hyperparameter values we want to use. There is not much to choose here. We have to decide on the parameter schemes for the epislon-greedy policy and the learning rate. Finally we need to decide how long we want to train for, how many times we want to train and how long we want to evalute for.

> *\_start* indicates the starting value of the parameter.
>
> *\_end* indicates the final value of the parameter.
>
> *\_cutoff* indicates when the final value is reached, i.e. 0.5 means after 50% of the training.

**Note:** We don't use exploring starts in this setting. See our **[report](https://github.com/KodAgge/Reinforcement-Learning-for-Market-Making/blob/main/Reinforcement%20Learning%20for%20Market%20Making.pdf)** for the reason why.

In [3]:
Q_learning_params = {
        # epsilon-greedy values (linear decay)
        "epsilon_start": 1,
        "epsilon_end": 0.05,
        "epsilon_cutoff": 0.5,

        # learning-rate values (exponential decay)
        "alpha_start": 0.5,
        "alpha_end": 0.001,
}

hyperparams = {
        "n_train" : 3e3,
        "n_test" : 3e2,
        "n_runs" : 4
}

Finally we decide where to save our results.

In [4]:
# naming the folder where the results will be saved
folder_mode = True
folder_name = "mc_example"
save_mode = True

We're now ready for the Q-learning!

This is easily done with the function *Q\_learning\_comparison*.

In [5]:
Q_learning_comparison(
    **hyperparams,
    args=model_params,
    Q_learning_args=Q_learning_params,
    folder_mode = folder_mode,
    folder_name = folder_name,
    save_mode = save_mode
)

RUN 1 IN PROGRESS...
	Episode 600 (20%), 0:04:02.470000 remaining of this run
	Episode 1200 (40%), 0:03:00.580000 remaining of this run
	Episode 1800 (60%), 0:01:59.500000 remaining of this run
	Episode 2400 (80%), 0:00:59.630000 remaining of this run
	Episode 3000 (100%), 0:00:00 remaining of this run
THE FOLDER mc_example ALREADY EXISTS
...FINISHED IN 0:04:58.260000
0:14:54.790000 REMAINING OF THE TRAINING
RUN 2 IN PROGRESS...
	Episode 600 (20%), 0:03:58.930000 remaining of this run
	Episode 1200 (40%), 0:02:58.210000 remaining of this run
	Episode 1800 (60%), 0:01:56.890000 remaining of this run
	Episode 2400 (80%), 0:00:58.010000 remaining of this run
	Episode 3000 (100%), 0:00:00 remaining of this run
THE FOLDER mc_example ALREADY EXISTS
...FINISHED IN 0:04:47.830000
0:09:35.650000 REMAINING OF THE TRAINING
RUN 3 IN PROGRESS...
	Episode 600 (20%), 0:03:52.790000 remaining of this run
	Episode 1200 (40%), 0:02:56.420000 remaining of this run
	Episode 1800 (60%), 0:01:59.340000 rema

## Evaluating the strategies

We can now have a look at the images that were saved when running *Q\_learning\_comparison*.

Let's first have a look at the reward and the state-value at (0,0) during training.

<div>
    <img src="results/mc_model/mc_example/results_graph.png"/>
</div>

In this image it looks like that the Q-learning has converged, however, it has not. It has to be trained for *much* longer. This will become very evident from the images below.

We can also have a look the learnt strategies. The figure below shows the learnt bid depths.

<div>
    <img src="results/mc_model/mc_example/opt_bid_heat.png" width="500"/>
</div>

Furthermore, we can compare the average rewards of the Q-learning strategies versus some benchmarking strategies. These are displayed in the boxplot below.

<div>
    <img src="results/mc_model/mc_example/box_plot_benchmarking.png"/>
</div>

We can also view these results in table form.


In [6]:
f = open("results/mc_model/mc_example/table_benchmarking")
print(f.read())
f.close()

strategy                 mean reward    std reward
---------------------  -------------  ------------
constant (d=1)              1.77           8.81346
random                     -0.423333       7.72123
Q_learning (best run)      -0.48           5.63941
Q_learning (average)       -0.79           5.94019


From the results it's clear that we have to train for much longer before we can find any good strategies. Interestingly the constant strategy severly outperforms all other strategies.

## More results?

There are a lot more figures and tables to explore which can be found in the **[mc_example](https://github.com/KodAgge/Reinforcement-Learning-for-Market-Making/tree/main/code/results/mc_model/mc_example)** folder.